<a href="https://colab.research.google.com/github/kimngan2809/AIproject/blob/main/RepresentationLearningSignatureVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
cur_dir = '/content/drive/My Drive/Development/'
os.chdir(cur_dir)
!ls -la .

ls: data_signature: No such file or directory
total 98261
-rw------- 1 root root      176 Sep 14 19:52 'Chỉnh data.gsheet'
lrw------- 1 root root        0 Aug 26 19:33  data_signature -> /content/drive/.shortcut-targets-by-id/1cJqRJxTwR9rixYs3ooCqHD_jUojoGC9b/data_signature
-rw------- 1 root root   664434 Sep 16 16:16 'DL Signature .ipynb'
-rw------- 1 root root    11723 Sep 15 12:47  pre_prepared_data.ipynb
-rw------- 1 root root    88209 Sep 23 09:08 'Preprocessing DL Signature .ipynb'
-rw------- 1 root root    28510 Sep 30 18:04  RepresentationLearningSignatureVerification.ipynb
drwx------ 2 root root     4096 Sep 30 18:04  saved_models
drwx------ 2 root root     4096 Sep 24 10:29  signature_pairs
drwx------ 2 root root     4096 Sep  8 01:58  sign_data
drwx------ 4 root root     4096 Sep 30 16:20  sign_data_kaggle_modified
-rw------- 1 root root     9797 Sep 24 15:27  Test.ipynb
-rw------- 1 root root 44759990 Sep 24 14:17  test_model.pth
-rw------- 1 root root   107389 Sep 24 10:3

**Loading Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics.pairwise import cosine_similarity
import os

**Image Loading and augumentations**

**Trained on a subset of data from [Kaggle Signature Dataset](https://www.kaggle.com/robinreni/signature-verification-dataset).** Took only 10 classes

In [ ]:
train_directory = cur_dir + 'sign_data_kaggle_modified/train'
test_directory = cur_dir + 'sign_data_kaggle_modified/test'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0.1,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.1
)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size = (224, 224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size=32

)


test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size = (224, 224),
    color_mode = 'rgb',
    shuffle = True,
    batch_size=32
)

Found 200 images belonging to 10 classes.
Found 40 images belonging to 10 classes.


**Import pretrained Models**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

vgg_basemodel = VGG16(include_top=True)
resnet_basemodel = ResNet50(include_top=True)

**Defining Callbacks**

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
# checkpoint = ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

**VGG16 Feature extractor**  
Removing the last layer and add a 10 class softmax layer (I took only 10 classes from the kaggle dataset, I recommend you to train it with the whole dataset for the best results)

In [ ]:
vgg_model = tf.keras.Sequential(vgg_basemodel.layers[:-1])
vgg_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# Freezing original layers
for layer in vgg_model.layers[:-1]:
    layer.trainable = False

vgg_model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.001, decay=0.01),
                  metrics=['accuracy'])

history = vgg_model.fit(train_generator,
              epochs=30,
              batch_size=64,
              validation_data=test_generator,
              callbacks=[early_stopping])

# finetuning with all layers set trainable

for layer in vgg_model.layers:
    layer.trainable = True

vgg_model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.0001),
                  metrics=['accuracy'])

history2 = vgg_model.fit(train_generator,
              epochs=30,
              batch_size=64,
              validation_data=test_generator,
              callbacks=[early_stopping])



Epoch 1/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.0571 - loss: 2.5966 - val_accuracy: 0.1000 - val_loss: 2.3725
Epoch 2/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 244ms/step - accuracy: 0.1023 - loss: 2.4562 - val_accuracy: 0.2750 - val_loss: 2.2949
Epoch 3/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - accuracy: 0.1761 - loss: 2.3580 - val_accuracy: 0.2000 - val_loss: 2.1464
Epoch 4/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 235ms/step - accuracy: 0.1958 - loss: 2.1832 - val_accuracy: 0.4500 - val_loss: 1.9437
Epoch 5/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 197ms/step - accuracy: 0.3256 - loss: 1.9699 - val_accuracy: 0.5000 - val_loss: 1.8076
Epoch 6/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - accuracy: 0.4058 - loss: 1.8941 - val_accuracy: 0.6000 - val_loss: 1.6820
Epoch 7/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 209ms/step - accuracy: 0.4014 - loss: 1.8250 - val_accuracy: 0.4750 - val_loss: 1.6129
Epoch 8/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 200ms/step - accuracy: 0.4272 - loss: 1.7072 - val_accuracy: 0.7500 - val_loss: 

In [ ]:
vgg_model.save(cur_dir + 'saved_models/vgg16_finetuned_model.h5')

NameError: name 'vgg_model' is not defined

**ResNet50 Feature extractor**  
The paper describes to use the output of second last convolutional layer of ResNet50 as a feature extractor. But didn't used the ResNet as I was getting good results with using VGG16 alone. I recommend you to try out this.

In [ ]:
resnet_basemodel.summary()

In [ ]:
# from tensorflow.python.keras.layers import Lambda;

# def lambda_layer(layers):

#     fc2_norm = K.l2_normalize(fc2, axis = 3);
#     illum_est = tf.reduce_sum(fc2_norm, axis = (1, 2));
#     illum_est = K.l2_normalize(illum_est);

#     return illum_est;

# illum_est = Lambda(norm)(fc2);


# input = resnet_basemodel.input
# x = tf.keras.layers.Flatten()(resnet_basemodel.get_layer('conv5_block3_2_conv').output)
# output = tf.keras.layers.Dense(10, activation='softmax')

# resnet_model = tf.keras.Model(inputs=[input], outputs=[output])

In [ ]:
# resnet_model = tf.keras.Sequential(resnet_basemodel.layers[:-1])

In [ ]:
# resnet_model = tf.keras.Sequential(resnet_basemodel.layers[-9])
# resnet_model.add(tf.keras.layers.Flatten())
# resnet_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# for layer in resnet_model.layers[:-9]:
#     layer.trainable = False

# resnet_model.compile(loss='categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.SGD(momentum=0.9, lr=0.001, decay=0.01),
#                   metrics=['accuracy'])

# history = resnet_model.fit(train_generator,
#               epochs=30,
#               batch_size=32,
#               validation_data=test_generator,
#               callbacks=[early_stopping])

# # finetuning with all layers set trainable

# for layer in resnet_model.layers:
#     layer.trainable = True

# resnet_model.compile(loss='categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.SGD(momentum=0.9, lr=0.0001),
#                   metrics=['accuracy'])

# history2 = resnet_model.fit(train_generator,
#               epochs=5,
#               batch_size=64,
#               validation_data=test_generator,
#               callbacks=[early_stopping])

# resnet_model.save('saved_models/resnet_finetuned_model')

**Loading Finetuned Model & Feature Extraction**  
[[1]](https://arxiv.org/abs/2004.12104) paper uses the output from the first fully connected layer of the VGG16 as the features for finding cosine similarity.

In [ ]:
# resnet_model = tf.keras.Sequential(resnet_basemodel.layers[-9])
# resnet_model.add(tf.keras.layers.Flatten())
# resnet_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
vgg_model = tf.keras.models.load_model(cur_dir + 'saved_models/vgg_finetuned_model.h5')
feature_extractor = tf.keras.Sequential(vgg_model.layers[:-1])

In [ ]:
print(vgg_model)

<Sequential name=sequential_1, built=True>
